In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as const
from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive
from astropy.coordinates import SkyCoord
import warnings
import pickle
warnings.filterwarnings('ignore')
def mass_radius(R):
    if R <= 1.23*u.earthRad:
        return(0.9718 * R.to('earthRad').value ** 3.58 * u.earthMass)
    elif R > 1.23*u.earthRad and R < 14.26*u.earthRad:
        return(1.436 * R.to('earthRad').value ** 1.7 * u.earthMass)
    else:
        return(0* u.earthMass)

Define observing constraints: 

In [4]:
# Equilibrium temperatures
teq_min = 800 * u.K
teq_max = 1500 * u.K

rad_min = 2.0 * u.R_earth
rad_max = 7.0 * u.R_earth
gaia_mag_limit = 15

JWST_mode = 'NIRSpec Prism'
#Available modes:
# NIRSpec Prism - NIRSpec G395M - NIRSpec G140M


Download NASA Exoplanet Archive table from the web via astroquery, select the part of the population that is transiting and save it to a pickle file for running Pandexo simulations with Pandexo.py. WARNING: Do not run this script while Pandexo.py is running!

In [3]:
table = NasaExoplanetArchive.get_confirmed_planets_table(all_columns=True)
transiting = table[(table['pl_tranflag'].data).astype(bool)]
pickle.dump( transiting, open( "table.p", "wb" ) )

After this line, you should run the accompanying script Pandexo.py to add Pandexo simulations of the SNR and observation time (minus overhead) as columns to the transiting planets table. This table is then read in again in the following line:

In [5]:
transiting = pickle.load(open("table_pandexo.p", "rb" ))

Compute equilibrium temperatures for zero albedo and the (relative) scale height: 

In [6]:
rp = transiting['pl_radj']
equilibrium_temperature = (transiting['st_teff'] * np.sqrt(transiting['st_rad'] / 2 / transiting['pl_orbsmax'])).decompose()
g = transiting['gaia_gmag'].quantity
transiting['teq'] = equilibrium_temperature

mean_molecular_mass = 6 * u.u#Assuming a default mean molecular weight. Unknown for almost all targets.
transiting['mass_predicted']=[mass_radius(x) for x in rp]

# gp = const.G * 10.0*u.earthMass / rp**2#Assuming a mass of 10Me
gp = const.G * transiting['mass_predicted'] / rp**2#Assuming a mass from the Chen et al. M-R. 
#Most of our targets have no masses. For the ones that do, we'd need to take the difference in H into account by eye.
transiting['H'] = ((const.k_B * equilibrium_temperature) / (mean_molecular_mass * gp)).to(u.km)
transiting['signal'] = (2*rp*transiting['H']/transiting['st_rad']**2).decompose()
transiting[JWST_mode+'_error'][transiting[JWST_mode+'_error']<0]=np.nan#Set planets that were skipped by Pandexo to NaN.
transiting['Kempton_metric'] = rp**3 / transiting['mass_predicted'] *equilibrium_temperature/transiting['st_rad']**2 / 10**(transiting['st_j']/5.0)

Apply the selection constraints and prepare for output:

In [7]:
temp_constraints = (equilibrium_temperature < teq_max) & (equilibrium_temperature > teq_min)
rad_constraints = (rp < rad_max) & (rp > rad_min)
gmag_constaints = (g < gaia_mag_limit)
targets = transiting[rad_constraints & temp_constraints & gmag_constaints]#This is the table that will be plotted.
targets.sort('st_j')
targets['r_earth'] = targets['pl_radj'].to(u.R_earth)
targets['Teq']=np.round(targets['teq'],0)
targets['Rp']=np.round(targets['r_earth'],2)
targets['H (predicted)']=np.round(targets['H'],0)
targets['Orbital Period']=np.round(targets['pl_orbper'],2)
targets['DEC']=np.round(targets['dec'],1)
targets['dur']=np.round(targets['pl_trandur']*24.0,1)*u.h
targets['H_ppm']=np.round(1e6*targets['signal'],2)
targets['JWST_ppm']=np.round(1e6*targets[JWST_mode+'_error'],1)
targets['SNR_H']=np.round(targets['signal']/targets['NIRSpec G395M_error'],4)
targets['TSM (J)'] = np.round(targets['Kempton_metric'].value*100,1)
targets['T_eff']=np.round(targets['st_teff'],0)
targets['Mass (measured)']=np.round(targets['pl_bmasse'],2)*u.earthMass
targets['Mass (predicted)']=np.round(targets['mass_predicted'],2)
targets[['pl_name', 'st_j', 'Teq', 'Rp', 'JWST_ppm' ,'Mass (measured)','Mass (predicted)','Orbital Period','H (predicted)','H_ppm','SNR_H','TSM (J)','dur','DEC','T_eff','st_spstr']].show_in_notebook()

idx,pl_name,st_j,Teq,Rp,JWST_ppm,Mass (measured),Mass (predicted),Orbital Period,H (predicted),H_ppm,SNR_H,TSM (J),dur,DEC,T_eff,st_spstr
,,,K,earthRad,,earthMass,earthMass,d,km,,,,h,deg,K,
0,pi Men c,4.869,1167.0,2.04,14.8,4.82,4.83,6.27,142.0,6.33,0.1333,12.8,3.0,-80.5,6037.0,G0 V
1,HR 858 d,5.473,1068.0,2.16,18.3,0.0,5.33,11.23,133.0,4.4,0.0748,6.7,3.4,-30.8,6201.0,F6 V
2,HAT-P-11 b,7.608,829.0,4.36,nan,23.4,17.55,4.89,127.0,31.57,-0.0,18.1,0.0,48.1,4780.0,K4
3,HD 106315 c,8.116,888.0,4.35,54.0,15.2,17.48,21.06,136.0,9.21,0.0637,4.2,4.6,-0.4,6327.0,F5 V
4,HD 106315 b,8.116,1155.0,2.44,59.8,12.6,6.56,9.55,149.0,5.67,0.0353,2.6,3.8,-0.4,6327.0,F5 V
5,Kepler-410 A b,8.417,1004.0,2.84,nan,0.0,8.45,17.83,135.0,5.54,-0.0,2.2,0.0,45.1,6273.0,--
6,K2-222 b,8.423,876.0,2.39,nan,0.0,6.3,15.39,112.0,5.94,-0.0,2.3,0.0,11.8,6058.0,--
7,HD 219666 b,8.557,1073.0,4.71,91.8,16.6,20.0,6.04,168.0,19.67,0.0891,7.3,2.2,-56.9,5527.0,--
8,TOI-1130 b,9.055,812.0,3.65,95.8,0.0,13.0,4.07,118.0,23.87,0.1139,7.0,2.3,-41.4,4250.0,K7


In [15]:
#table[['pl_name','pl_orbper']].show_in_notebook() #This outputs the entire table of exoplanets.